# 1

In [3]:
class Mymeta(type): # 只有继承了type类才能称之为一个元类，否则就是一个普通的自定义类
    def __call__(self,*args,**kwargs):
        print('self:\t',self)
        print('args:\t',args)
        print('kwargs:\t',kwargs)
        return 123
    
class OldboyTeacher(object,metaclass=Mymeta):
    school = 'oldboy'
    
    def __init__(self,name,age):
        self.name = name
        self.age = age
        
    def say(self):
        print('%s says welcome to the oldboy to learn Python' %self.name)

In [6]:
# 调用OldboyTeacher就是在调用OldboyTeacher类中的__call__方法
# 然后将OldboyTeacher传给self,溢出的位置参数传给*，溢出的关键字参数传给**
# 调用OldboyTeacher的返回值就是调用__call__的返回值

t1 = OldboyTeacher('egon',18,name='zhangsan',age=20)
t1

self:	 <class '__main__.OldboyTeacher'>
args:	 ('egon', 18)
kwargs:	 {'name': 'zhangsan', 'age': 20}


123

# 2

In [7]:
class Mymeta(type):
    def __call__(self, *args, **kwargs):
        # 1、调用__new__产生一个空对象obj
        # 此处的self是类OldoyTeacher，必须传参，代表创建一个OldboyTeacher的对象obj
        obj = self.__new__(self)
        
        # 2、调用__init__初始化空对象obj
        self.__init__(obj,*args,**kwargs)
        
        # 3、返回初始化好的对象obj
        return obj

class OldboyTeacher(object,metaclass=Mymeta):
    school = 'oldboy'
    
    def __init__(self,name,age):
        self.name = name
        self.age = age
        
    def say(self):
        print('%s says welcome to the oldboy to learn Python' %self.name)

t1 = OldboyTeacher('egon',18)
print(t1.__dict__)

{'name': 'egon', 'age': 18}


# 3

In [8]:
class Mymeta(type):
    def __call__(self,*args,**kwargs):
        # 1.调用__new__产生一个空对象Obj
        obj = self.__new__(self)
        
        # 2.调用__init__初始化空对象obj
        self.__init__(obj,*args,**kwargs)
        
        # 在初始化之后,obj.__dict__里就有值了
        obj.__dict__ = {'_%s__%s'%(self.__name__,k):v for k,v in obj.__dict__.items()}
        
        # 3.返回初始化好的对象obj
        return obj
    
class OldboyTeacher(object,metaclass=Mymeta):
    school = 'oldboy'
    
    def __init__(self,name,age):
        self.name = name
        self.age = age
        
    def say(self):
        print('%s says welcome to the oldboy to learn Python' %self.name)
        
t1 = OldboyTeacher('egon',18)
t1.__dict__

{'_OldboyTeacher__name': 'egon', '_OldboyTeacher__age': 18}

# metaclass创建自己的orm

In [5]:
"""
当我们定义了类以后，就可以根据这个类创建出实例，所以：先定义类，然后创建实例。

但是如果我们想创建出类呢？那就必须根据metaclass创建出类，所以：先定义metaclass，然后创建类。

连接起来就是：先定义metaclass，就可以创建类，最后创建实例。

所以，metaclass允许你创建类或者修改类。换句话说，你可以把类看成是metaclass创建出来的“实例”。

metaclass是Python面向对象里最难理解，也是最难使用的魔术代码。
正常情况下，你不会碰到需要使用metaclass的情况，所以，以下内容看不懂也没关系，因为基本上你不会用到。

我们先看一个简单的例子，这个metaclass可以给我们自定义的MyList增加一个add方法：

定义ListMetaclass，按照默认习惯，metaclass的类名总是以Metaclass结尾，以便清楚地表示这是一个metaclass：
"""

class ListMetaclass(type):
    def __new__(cls,name,bases,attrs):
        attrs["add"] = lambda self,value:self.append(value)
        return type.__new__(cls,name,bases,attrs)
    
"有了ListMetaclass，我们在定义类的时候还要指示使用ListMetaclass来定制类，传入关键字参数metaclass："
class MyList(list,metaclass=ListMetaclass):
    pass

In [9]:
"""
当我们传入关键词参数metaclass时，魔术就生效了，它指示Python解释器在创建MyList时，要通过ListMetaclass.__new__()来创建，
在此，我们可以修改类的定义，比如，加上新的方法，然后，返回修改后的定义

__new__() 方法接收到的参数依次是：
1.当前准备创建的类的对象
2.类的名字
3.类继承的父类集合
4.类的方法集合
"""

# 测试一下MyList是否可以调用add() 方法
L = MyList()
L.add(1)
print(L)

# 而普通的list没有add方法
L2 = list()
L2.add(1)

[1]


AttributeError: 'list' object has no attribute 'add'

In [11]:
class Field(object):
    def __init__(self,name,column_type):
        self.name = name
        self.column_type = column_type
    
    def __str__(self):
        return "<%s:%s>"%(self.__class__.__name__,self.name)

# 在Field的基础上，进一步定义各种类型的field 比如stringField，IntegerField等等
class StringField(Field):
    def __init__(self,name):
        super(StringField,self).__init__(name,"varchar(100)")
        
class IntegerField(Field):
    def __init__(self,name):
        super(IntegerField,self).__init__(name,"bigint")
    
# 下一步，就是编写最复杂的ModelMetaclass了:
class ModelMetaclass(type):
    def __new__(cls,name,bases,attrs):
        if name == "Model": # Model直接返回创建类  非Model类需要遍历
            return type.__new__(cls,name,bases,attrs)
        print("Found Model:%s"%name)
        mappings = dict()
        for k,v in attrs.items():
            if isinstance(v,Field):
                print("Found mapping%s===>%s"%(k,v))
                mappings[k] = v
        for k in mappings:
            attrs.pop(k)
        
        attrs["__mappings__"] = mappings # 保存属性和列的映射关系
        attrs["__table__"] = name # 假设表名和类名一致
        return type.__new__(cls,name,bases,attrs)
    
# 以及基类Model:
class Model(dict,metaclass=ModelMetaclass):
    def __init__(self,**kw):
        super(Model,self).__init__(**kw)
    
    def __getattr__(self,key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"Model object has no attribute %s"%key)
    
    def __setattr__(self,key,value):
        self[key] = value # 创建类时已经将key value映射
    
    def save(self):
        fields = []
        params = []
        args = []
        for k,v in self.__mappings__.items():
            fields.append(v.name)
            params.append("?")
            args.append(getattr(self,k,None))
        sql = "insert into %s(%s) values(%s)"%(self.__table__,",".join(fields),",".join(params))
        print("SQL:%s"%sql)
        print("ARGS:%s"%str(args))

In [17]:
"""
当用户定义一个class User(Model)时，Python解释器首先在当前类User的定义中查找metaclass，
如果没有找到，就继续在父类Model中查找metaclass，找到了，
就使用Model中定义的metaclass的ModelMetaclass来创建User类，
也就是说，metaclass可以隐式地继承到子类，但子类自己却感觉不到。

在ModelMetaclass中，一共做了几件事情：

1.排除掉对Model类的修改；

2.在当前类（比如User）中查找定义的类的所有属性，
如果找到一个Field属性，就把它保存到一个__mappings__的dict中，
同时从类属性中删除该Field属性，否则，容易造成运行时错误（实例的属性会遮盖类的同名属性）；

3.把表名保存到__table__中，这里简化为表名默认为类名。

在Model类中，就可以定义各种操作数据库的方法，比如save()，delete()，find()，update等等。

我们实现了save()方法，把一个实例保存到数据库中。因为有表名，属性到字段的映射和属性值的集合，就可以构造出INSERT语句。
"""

class User(Model):
    # 定义类的属性到列的映射
    id = IntegerField("id")
    name = StringField("username")
    email = StringField("email")
    password = StringField("password")
    
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
u.save()

"""
执行过程
1.metaclass先创建Model类
2.以此创建Field各个子类
3.metaclass创建User类 User类中的attrs对应的是各个field的key value 如下：
'id':<__main__.IntegerField object at 0x10d6f2790>
'name':<__main__.StringField object at 0x10db8db10>
'email':<__main__.StringField object at 0x10d211450>
'password':<__main__.StringField object at 0x10e5eec50>
"""

Found Model:User
Found mappingLid===><IntegerField:id>
Found mappingLname===><StringField:username>
Found mappingLemail===><StringField:email>
Found mappingLpassword===><StringField:password>
SQL:insert into User(id,username,email,password) values(?,?,?,?)
ARGS:[12345, 'Michael', 'test@orm.org', 'my-pwd']
